Input filter parameters


In [31]:
BPE_token_limit=128
Text_Min_Len=5
Text_Max_Len=2048
Check_For_URL=False
Check_For_Text_Len=False
Check_For_ASCII=False
Check_For_Deleted=False
Check_For_BPE=False

importing libraries

In [32]:
import json
import numpy as np
import re 
from collections import Counter, defaultdict


Create URL filter


In [33]:
def CheckNoURL(string): 
    # findall() has been used  
    # with valid conditions for urls in string 
    #regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    #url = re.findall(regex,string) 
    return ('http' in string) or ('https' in string) or ('www' in string)

Create BPE token filter

In [34]:
def build_vocab(corpus: str) -> dict:
    """Step 1. Build vocab from text corpus"""

    # Separate each char in word by space and add mark end of token
    tokens = [" ".join(word) + " </w>" for word in corpus.split()]
    
    # Count frequency of tokens in corpus
    vocab = Counter(tokens)  

    return vocab


def get_stats(vocab: dict) -> dict:
    """Step 2. Get counts of pairs of consecutive symbols"""

    pairs = defaultdict(int)
    for word, frequency in vocab.items():
        symbols = word.split()

        # Counting up occurrences of pairs
        for i in range(len(symbols) - 1):
            pairs[symbols[i], symbols[i + 1]] += frequency

    return pairs


def merge_vocab(pair: tuple, v_in: dict) -> dict:
    """Step 3. Merge all occurrences of the most frequent pair"""
    
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    
    for word in v_in:
        # replace most frequent pair in all vocabulary
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]

    return v_out

def GetBPETokens(input_txt):
 try:
   vocab = build_vocab(input_txt)  # Step 1
   num_merges = 40  # Hyperparameter
   for i in range(num_merges):
    pairs = get_stats(vocab)  # Step 2
    if not pairs:
        break
    # step 3
    best = max(pairs, key=pairs.get)
    vocab = merge_vocab(best, vocab)
   return (len(vocab)>BPE_token_limit)
 except:
    return False


Create text length constrain filter

In [35]:
def TextLenFilter(input_text):
  if len(input_text)>Text_Max_Len or len(input_text)<Text_Min_Len:
     return True
  else:
     return False

Create ASCII checker

In [36]:
def CheckForASCII(input_text):
  if input_text.isascii():
     return False
  else:
     return True


Check for deleted comments


In [37]:
def CheckForDelete(input_text):
  if input_text in ('[deleted]','[removed]'):
     return True
  else:
     return False

All filters combined


In [38]:
def FiltTxt(input_text):
  if Check_For_Deleted and CheckForDelete(input_text):
    return False
  if Check_For_ASCII and CheckForASCII(input_text):
    return False
  if Check_For_Text_Len and TextLenFilter(input_text):
    return False
  if Check_For_URL and CheckNoURL(input_text):
    return False
  if Check_For_BPE and GetBPETokens(input_text):
    return False
  return True

Start filtering


In [39]:
episode_texts = []
episode_labels = []
episode_dones = []
with open('reddit_female.json') as json_file:
    data = json.load(json_file)
    for submission in data:
        if len(submission['comments']) == 0:
           continue
        else:
### Enter Level 1
          episode_text = []
          episode_label = []
          episode_done = []   
          title = submission['title']
          for comment in submission['comments']:
              comment_body = comment['body']
              if FiltTxt(comment['body']):  
                if len(comment['replies']) == 0:
                   episode_text.append(title)
                   episode_label.append(comment_body)
                   episode_done.append(True)
                else:
### Enter level 2
                   for reply in comment['replies']:
                       if FiltTxt(reply['body']):
                          if len(reply['replies']) == 0:
                             episode_text.append(title)
                             episode_label.append(comment_body)
                             episode_done.append(False)
                             episode_text.append(reply['body'])
                             episode_label.append('')
                             episode_done.append(True)
                          else:
### Enter level 3          
                            for reply1 in reply['replies']:
                                if FiltTxt(reply1['body']):
                                   if len(reply1['replies']) == 0:
                                      episode_text.append(title)
                                      episode_label.append(comment_body)
                                      episode_done.append(False)
                                      episode_text.append(reply['body'])
                                      episode_label.append(reply1['body'])
                                      episode_done.append(True) 
                                   else:
### Enter level 4                       
                                     for reply2 in reply1['replies']:
                                        if FiltTxt(reply2['body']):
                                          if len(reply2['replies']) == 0:
                                              episode_text.append(title)
                                              episode_label.append(comment_body)
                                              episode_done.append(False)
                                              episode_text.append(reply['body'])
                                              episode_label.append(reply1['body'])
                                              episode_done.append(False)
                                              episode_text.append(reply2['body'])
                                              episode_label.append('')
                                              episode_done.append(True) 
                                          else:
### Enter level 5                             
                                           for reply3 in reply2['replies']:
                                               if FiltTxt(reply3['body']):
                                                   if len(reply3['replies']) == 0:
                                                      episode_text.append(title)
                                                      episode_label.append(comment_body)
                                                      episode_done.append(False)
                                                      episode_text.append(reply['body'])
                                                      episode_label.append(reply1['body'])
                                                      episode_done.append(False)
                                                      episode_text.append(reply2['body'])
                                                      episode_label.append(reply3['body'])
                                                      episode_done.append(True) 
                                                   else:
### Enter level 6                                       
                                                    for reply4 in reply3['replies']:
                                                     if FiltTxt(reply4['body']):
                                                        if len(reply4['replies']) == 0:
                                                           episode_text.append(title)
                                                           episode_label.append(comment_body)
                                                           episode_done.append(False)
                                                           episode_text.append(reply['body'])
                                                           episode_label.append(reply1['body'])
                                                           episode_done.append(False)
                                                           episode_text.append(reply2['body'])
                                                           episode_label.append(reply3['body'])
                                                           episode_done.append(False)
                                                           episode_text.append(reply4['body'])
                                                           episode_label.append('')
                                                           episode_done.append(True) 
                                                        else:
### Enter level 7                                       
                                                           for reply5 in reply4['replies']:
                                                               if FiltTxt(reply5['body']):
                                                                     episode_text.append(title)
                                                                     episode_label.append(comment_body)
                                                                     episode_done.append(False)
                                                                     episode_text.append(reply['body'])
                                                                     episode_label.append(reply1['body'])
                                                                     episode_done.append(False)
                                                                     episode_text.append(reply2['body'])
                                                                     episode_label.append(reply3['body'])
                                                                     episode_done.append(False)
                                                                     episode_text.append(reply4['body'])
                                                                     episode_label.append(reply5['body'])
                                                                     episode_done.append(True)
                                                               else:
                                                                  continue
                                                     else:
                                                         continue              
                                               else:
                                                  continue             
                                        else:
                                              continue                
                                else:
                                   continue
                       else:
                         continue



              else:
                continue
          episode_texts.append(episode_text)
          episode_labels.append(episode_label)
          episode_dones.append(episode_done)

In [41]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [42]:
rows = pd.DataFrame()

In [43]:
flat_texts = [item for sublist in episode_texts for item in sublist]
flat_labels = [item for sublist in episode_labels for item in sublist]
flat_dones = [item for sublist in episode_dones for item in sublist]

In [44]:
rows['texts'] = flat_texts
rows['labels'] = flat_labels
rows['dones'] = flat_dones

In [45]:
with open('output_parlai_female.txt', 'w') as f:
    for idx, row in rows.iterrows():
        t = ("text:{}\tlabels:{}").format(row['texts'].replace('\n', ' '), row['labels'].replace('\n', ' '))
        if row['dones'] == True:
            t = t + '\tepisode_done:True\n'
        else:
            t = t + '\n'
        f.write("%s" % t)